In [1]:
import pandas as pd
pd.set_option("display.max_columns", None)
import numpy as np
import os

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

In [2]:
root = "/Volumes/TOB_WD2/Data_Analysis/DataFrames" + "/"
lineageDir = root + "Tracking/Output_Lineages" + "/"

lineages = []

for subdir, dirs, files in os.walk(lineageDir):
            for file in files:
                if not file.startswith('.'):
                    filepath = subdir + os.sep + file  
                    if filepath.endswith(".csv"):
                        single_df = pd.read_csv(filepath)
                        lineages.append(single_df)    

lineage_df = pd.concat(lineages)  
lineage_df = lineage_df.loc[:, ~lineage_df.columns.str.contains('^Unnamed')]

df = pd.read_csv(root + "MasterDataFrame_Population_Morphometry.csv")

df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
lineage_df.tail(5)

,Track_ID,Source_ID,Splitting_event,Lineage,Track_Coordinate_X,Track_Coordinate_Y,Frame,Timepoint,Position,Experiment,Dataset,Generation,Mother_ID,Grandmother_ID,Sister_ID,Seen_sister,Aunt_ID,Cousin_ID,Random_ID,Seen_granny
54,25,9558,True,9558,295.377,85.530,4,5,2,20200728-174144,20200728,1,NaN,NaN,NaN,NaN,NaN,NaN,8838,NaN
55,25,9569,True,9558.96,312.431,109.655,82,83,2,20200728-174144,20200728,2,9558.0,NaN,9594.0,True,NaN,NaN,9075,NaN
56,25,9594,True,9558.96,356.106,80.123,87,88,2,20200728-174144,20200728,2,9558.0,NaN,9569.0,False,NaN,NaN,9483,NaN
57,26,9618,True,9618,294.961,187.438,27,28,2,20200728-174144,20200728,1,NaN,NaN,NaN,NaN,NaN,NaN,9010,NaN
58,26,9631,True,9618.96,344.043,158.737,88,89,2,20200728-174144,20200728,2,9618.0,NaN,NaN,NaN,NaN,NaN,8637,NaN


In [3]:
def convert_TMcoordinates(x, coord):
    # convert uncalibrated trackmate coordinates to calibrated ones
    if not (x.Dataset == 20190618) & (x.Position == 2):
        if coord == "X":
            return x.Track_Coordinate_X
        if coord == "Y":
            return x.Track_Coordinate_Y
    else:
        px_per_micron = 1.61
        if coord == "X":
            Track_Coordinate_X_calibrated = x.Track_Coordinate_X / px_per_micron
            return Track_Coordinate_X_calibrated
        else:
            Track_Coordinate_Y_calibrated = x.Track_Coordinate_Y / px_per_micron
            return Track_Coordinate_Y_calibrated

lineage_df["Track_Coordinate_X"] = lineage_df.apply(convert_TMcoordinates, coord = "X", axis = 1)
lineage_df["Track_Coordinate_Y"] = lineage_df.apply(convert_TMcoordinates, coord = "Y", axis = 1)

In [4]:
lineage_df.head()

,Track_ID,Source_ID,Splitting_event,Lineage,Track_Coordinate_X,Track_Coordinate_Y,Frame,Timepoint,Position,Experiment,Dataset,Generation,Mother_ID,Grandmother_ID,Sister_ID,Seen_sister,Aunt_ID,Cousin_ID,Random_ID,Seen_granny
0,0,199492,True,199492,193.885,442.832,42,43,1,20200724-201611,20200724,1,NaN,NaN,NaN,NaN,NaN,NaN,206918,NaN
1,0,199509,True,199492.199509,86.570,497.113,99,100,1,20200725-142832,20200724,2,199492.0,NaN,NaN,NaN,NaN,NaN,204363,NaN
2,0,199522,True,199492.199509.199522,130.869,469.037,167,168,1,20200725-142832,20200724,3,199509.0,199492.0,199580.0,False,NaN,NaN,202241,False
3,0,199580,True,199492.199509.199580,143.971,453.438,167,168,1,20200725-142832,20200724,3,199509.0,199492.0,199522.0,True,NaN,NaN,208123,True
4,0,199602,True,199492.199509.199580.199602,144.283,416.627,229,230,1,20200725-142832,20200724,4,199580.0,199509.0,199640.0,True,199522.0,199566,201486,True


In [5]:
def convert_timepoints(x):
    # adjust timepoints for interrupted experiments CHECK 0618!!!
    if x.Experiment == "20200725-142832":
        return x.Timepoint + 93
    elif x.Experiment == "20200802-104739":
        return x.Timepoint + 225
    elif x.Experiment == "20200803-181830":
        return x.Timepoint + 225 + 149
    elif x.Experiment == "20200809-114236":
        return x.Timepoint + 199
    elif x.Experiment == "20200810-225504":
        return x.Timepoint + 199 + 179
    else:
        return x.Timepoint

def convert_coordinates(x, coord):
    # convert pixel coordinates to um
    if x.Experiment == "20190618-conc":
        px_per_micron = 1.61
        if coord == "X":
            X_um = x.X_px / px_per_micron
            return X_um
        else:
            Y_um = x.Y_px / px_per_micron
            return Y_um
    else:
        px_per_micron = 3.22
        if coord == "X":
            X_um = x.X_px / px_per_micron
            return X_um
        else:
            Y_um = x.Y_px / px_per_micron
            return Y_um
        


        
df["X_um"] = df.apply(convert_coordinates, coord = "X", axis = 1)
df["Y_um"] = df.apply(convert_coordinates, coord = "Y", axis = 1)
df["Timepoint"] = df.apply(convert_timepoints, axis = 1)

df.head()

,Cell_ID,LowZoom_ID,Dataset,Experiment,Position,Condition,Has_duplicate,X_px,Y_px,Timepoint,Time,Differentiation_mins,Differentiation_bins,Experiment_Time_mins,index,Chromatin_Dilation,Chromatin_Volume_um3,DNA_Volume_Threshold,MetaphasePlate_Length_um,MetaphasePlate_Width_um,Spindle_Angle_Degrees,Spindle_Aspect_Ratio,Spindle_DNA_Volume_Ratio,Spindle_Length_um,Spindle_Volume_um3,Spindle_Width_Avg_um,Tubulin_Spindle_Average_Intensity,Version,Cell_Volume_um3,SurfaceArea,Sphericity,Tubulin_Cell_Average,Tubulin_Cell_Minimum,Tubulin_Cell_Maximum,Tubulin_Cell_IntDen,Tubulin_mass_cell,Tubulin_mass_cell_norm,Tubulin_mass_spindle,Tubulin_mass_spindle_norm,Tubulin_mass_cytoplasm,Tubulin_mass_cytoplasm_norm,Tubulin_Cytop_Average,Tubulin_density_cytop_norm,Fraction_Tubulin_in_Spindle,Fraction_SpindleVol_in_Cell,Tubulin_Averages_Ratio,Tubulin_Averages_Cytop_Ratio,Tubulin_density_spindle_norm,SpindleVolume_ChromatinVolume_Ratio,CellSurfaceArea_CellVolume_Ratio,Cell_Volume_bin,CellSurfaceArea_CellVolume_bin,Spindle_Volume_bin,Spindle_Width_bin,Colour,States,Fraction_Tubulin_in_Spindle_P1mean,Fraction_Tubulin_in_Spindle_P1std,Fraction_Tubulin_in_Spindle_P2mean,Fraction_Tubulin_in_Spindle_P2std,Fraction_Tubulin_in_Spindle_BinNormP1,Fraction_SpindleVol_in_Cell_P1mean,Fraction_SpindleVol_in_Cell_P1std,Fraction_SpindleVol_in_Cell_P2mean,Fraction_SpindleVol_in_Cell_P2std,Fraction_SpindleVol_in_Cell_BinNormP1,Spindle_Volume_um3_P1mean,Spindle_Volume_um3_P1std,Spindle_Volume_um3_P2mean,Spindle_Volume_um3_P2std,Spindle_Volume_um3_BinNormP1,X_um,Y_um
0,20190227-182447_HighZoom--W0000--P0001-T0004--...,20190227-182447_LowZoom--W0000--P0001-T0004,20190227,20190227-182447,1,1_Pluripotent,False,173.120841,1222.898424,4,2019/02/27 18:55:28,450.016667,"(0, 2880]",30.016667,0.0,0.231457,230.203125,18.0,13.0,1.75,2.778096,1.317042,1.723749,10.761622,396.812500,8.171053,43.786896,Spindle Morphometry Version: 0.7.7,2712.703,949.315,0.973,13.560,0.0,99.0,2354144.0,36784.252680,2712.703,17375.187500,1281.356010,19409.065180,1431.346990,8.380822,0.618055,47.235396,14.627938,3.229122,3.229122,3.229122,1.723749,0.349952,"(2500.0, 3000.0]","(0.3, 0.35]","(350.0, 400.0]","(7.0, 9.0]",#E8C468,early,45.779758,4.347672,42.206494,4.347672,1.031797,12.717699,2.098323,11.268173,2.098323,1.150203,350.09424,59.658337,305.519395,59.658337,1.133445,53.764236,379.782119
1,20190227-182447_HighZoom--W0000--P0001-T0012--...,20190227-182447_LowZoom--W0000--P0001-T0012,20190227,20190227-182447,1,1_Pluripotent,False,946.163488,638.107629,12,2019/02/27 20:15:33,530.100000,"(0, 2880]",110.100000,0.0,0.638758,237.187500,17.0,12.0,2.75,25.920980,1.294125,1.206522,9.688911,286.171875,7.486842,79.449031,Spindle Morphometry Version: 0.7.7,2846.719,990.024,0.945,20.026,0.0,182.0,3648458.0,57008.394694,2846.719,22736.078125,1135.327980,34272.316569,1711.391020,13.384763,0.668369,39.881983,10.052691,3.967294,3.967294,3.967294,1.206522,0.347777,"(2500.0, 3000.0]","(0.3, 0.35]","(250.0, 300.0]","(7.0, 9.0]",#E8C468,early,45.779758,4.347672,42.206494,4.347672,0.871171,12.717699,2.098323,11.268173,2.098323,0.790449,350.09424,59.658337,305.519395,59.658337,0.817414,293.839592,198.170071
2,20190227-182447_HighZoom--W0000--P0001-T0036--...,20190227-182447_LowZoom--W0000--P0001-T0036,20190227,20190227-182447,1,1_Pluripotent,False,794.802998,1335.670236,36,2019/02/28 00:15:45,770.300000,"(0, 2880]",350.300000,0.0,0.795803,215.531250,29.0,12.5,2.00,34.771214,1.340565,1.876106,11.183135,404.359375,8.342105,38.703737,Spindle Morphometry Version: 0.7.7,2666.109,943.153,0.958,11.794,0.0,105.0,2012371.0,31444.089546,2666.109,15650.218750,1326.964452,15793.870796,1339.144548,6.983032,0.592083,49.771575,15.166648,3.281646,3.281646,3.281646,1.876106,0.353756,"(2500.0, 3000.0]","(0.35, 0.4]","(400.0, 450.0]","(7.0, 9.0]",#E8C468,early,45.779758,4.347672,42.206494,4.347672,1.087196,12.717699,2.098323,11.268173,2.098323,1.192562,350.09424,59.658337,305.519395,59.658337,1.155002,246.833229,414.804421
3,20190227-18244

In [6]:
def find_match(x, trackmate, dataframes_list):
    # matching based on µm distances!
        cell_id = x.Cell_ID
        match = trackmate.loc[(trackmate.Position == x.Position) & (trackmate.Dataset == x.Dataset) &
                                (np.isclose(np.array(trackmate.Timepoint), np.array(x.Timepoint),\
                                              atol = 4.0) == np.array([ True ])) & 
                                (np.isclose(np.array(trackmate.Track_Coordinate_X), np.array(x.X_um),\
                                              atol = 10.0 ) == np.array([ True ])) & 
                                (np.isclose(np.array(trackmate.Track_Coordinate_Y), np.array(x.Y_um),\
                                              atol = 10.0 ) == np.array([ True ]))
                                ,:]
        if match.shape[0] > 0:
                match['Delta_x'] = match.Track_Coordinate_X.apply(lambda coord_x: abs(coord_x - x.X_um))
                match['Delta_y'] = match.Track_Coordinate_Y.apply(lambda coord_y: abs(coord_y - x.Y_um))
                match['Delta_t'] = match.Timepoint.apply(lambda time: abs(time - x.Timepoint))
                match.sort_values(by = ['Delta_t', 'Delta_x', 'Delta_y'], inplace = True)
                match = match.head(1)
                match["Cell_ID"] = cell_id
                dataframes_list.append(match)
        else:
            pass

matches_list = []
df.apply(find_match, trackmate = lineage_df, dataframes_list = matches_list, axis = 1)
matches = pd.concat(matches_list, axis = 0, ignore_index = True)
matches.shape

(1824, 24)

In [7]:
matches.head()

,Track_ID,Source_ID,Splitting_event,Lineage,Track_Coordinate_X,Track_Coordinate_Y,Frame,Timepoint,Position,Experiment,Dataset,Generation,Mother_ID,Grandmother_ID,Sister_ID,Seen_sister,Aunt_ID,Cousin_ID,Random_ID,Seen_granny,Delta_x,Delta_y,Delta_t,Cell_ID
0,25,6000,True,6000,54.334,381.167,3,4,1,20190227-182447,20190227,1,NaN,NaN,NaN,NaN,NaN,NaN,12985,NaN,0.569764,1.384881,0,20190227-182447_HighZoom--W0000--P0001-T0004--...
1,6,924,True,924,295.273,199.500,11,12,1,20190227-182447,20190227,1,NaN,NaN,NaN,NaN,NaN,NaN,7545,NaN,1.433408,1.329929,0,20190227-182447_HighZoom--W0000--P0001-T0012--...
2,20,2888,True,2888,247.543,416.315,35,36,1,20190227-182447,20190227,1,NaN,NaN,NaN,NaN,NaN,NaN,7789,NaN,0.709771,1.510579,0,20190227-182447_HighZoom--W0000--P0001-T0036--...
3,25,6010,True,6000.6010,6.707,376.799,76,77,1,20190227-182447,20190227,2,6000.0,NaN,6187.0,True,NaN,NaN,8518,NaN,0.267725,1.791010,0,20190227-182447_HighZoom--W0000--P0001-T0077--...
4,6,934,True,924.934,332.708,190.453,79,80,1,20190227-182447,20190227,2,924.0,NaN,1137.0,True,NaN,NaN,9004,NaN,0.244621,1.633552,1,20190227-182447_HighZoom--W0000--P0001-T0079--...


In [8]:
def matching_statistics(data, lineage_data, matches_data):
    list_of_datasets = lineage_data.Dataset.unique()
    data_count = data.groupby(["Dataset", "Position"]).Cell_ID.count().reset_index()
    lineage_count = lineage_data.groupby(["Dataset", "Position"]).Source_ID.count().reset_index()
    matches_count = matches_data.groupby(["Dataset", "Position"]).Cell_ID.count().reset_index()
    
    delta_x_avg = matches_data.groupby(["Dataset", "Position"]).Delta_x.mean().reset_index()
    delta_y_avg = matches_data.groupby(["Dataset", "Position"]).Delta_y.mean().reset_index()
    delta_t_avg = matches_data.groupby(["Dataset", "Position"]).Delta_t.mean().reset_index()

    dictionaries = []
    for dataset in list_of_datasets:
        list_of_positions = lineage_data[lineage_data.Dataset == dataset].Position.unique()
        for position in list_of_positions:
            no_of_cell_ids = data_count.loc[
                (data_count.Dataset == int(dataset)) & (data_count.Position == int(position)), "Cell_ID"
            ]
            if no_of_cell_ids.shape[0] > 0:
                no_of_cell_ids = no_of_cell_ids.item()
            else:
                no_of_cell_ids = 0
            
            no_of_splitting_events = lineage_count.loc[
                (lineage_count.Dataset == int(dataset)) & (lineage_count.Position == int(position)), "Source_ID"
            ]
            if  no_of_splitting_events.shape[0] > 0:
                no_of_splitting_events = no_of_splitting_events.item()
            else:
                no_of_splitting_events = 0
            
            no_of_matches = matches_count.loc[
                (matches_count.Dataset == int(dataset)) & (matches_count.Position == int(position)), "Cell_ID"
            ]
            if no_of_matches.shape[0] > 0:
                no_of_matches = no_of_matches.item()
            else:
                no_of_matches = 0
            
            average_delta_x = delta_x_avg.loc[
                (matches_count.Dataset == int(dataset)) & (matches_count.Position == int(position)), "Delta_x"
            ]
            if average_delta_x.shape[0] > 0:
                average_delta_x = average_delta_x.item()
            else:
                average_delta_x = None
                
            average_delta_y = delta_y_avg.loc[
                (matches_count.Dataset == int(dataset)) & (matches_count.Position == int(position)), "Delta_y"
            ]
            if average_delta_y.shape[0] > 0:
                average_delta_y = average_delta_y.item()
            else:
                average_delta_y = None
                
            average_delta_t = delta_t_avg.loc[
                (matches_count.Dataset == int(dataset)) & (matches_count.Position == int(position)), "Delta_t"
            ]
            if average_delta_t.shape[0] > 0:
                average_delta_t = average_delta_t.item()
            else:
                average_delta_t = None

            sub_dictionary = {"Dataset": dataset, 
                              "Position": position, 
                              "Number of Cell IDs": no_of_cell_ids, 
                              "Number of splitting events": no_of_splitting_events, 
                              "Number of matches": no_of_matches, 
                              "Maximum yield": no_of_cell_ids / no_of_splitting_events, 
                              "Matches per split event": no_of_matches / no_of_splitting_events, 
                              "Matches per cell id": no_of_matches / no_of_cell_ids, 
                              "Avg Delta x": average_delta_x, 
                              "Avg Delta y": average_delta_y, 
                              "Avg Delta t": average_delta_t}
            sub_df = pd.DataFrame(sub_dictionary, index = [0])
            dictionaries.append(sub_df)
    return pd.concat(dictionaries)
    
matching_stats = matching_statistics(data = df, lineage_data = lineage_df, matches_data = matches)

matching_stats.to_csv("/Volumes/TOB_WD2/Data_Analysis/DataFrames/MetaStatistics/MetaStatistics_CoordinateMatching.csv")
print("Finished saving the metastatistics for tracking.")

matching_stats

Finished saving the metastatistics for tracking.


,Dataset,Position,Number of Cell IDs,Number of splitting events,Number of matches,Maximum yield,Matches per split event,Matches per cell id,Avg Delta x,Avg Delta y,Avg Delta t
0,20200724,1,170,523,106,0.325048,0.202677,0.623529,1.849840,1.371177,2.575472
0,20200724,2,274,455,137,0.602198,0.301099,0.500000,3.154932,2.679137,0.481752
0,20190227,1,590,652,321,0.904908,0.492331,0.544068,1.339865,1.621744,0.457944
0,20190227,2,225,318,143,0.707547,0.449686,0.635556,1.686043,1.985933,0.699301
0,20190618,1,161,654,134,0.246177,0.204893,0.832298,1.491974,1.540668,0.320896
0,20190618,2,247,595,168,0.415126,0.282353,0.680162,1.254364,1.097626,0.589286
0,20190827,1,227,462,209,0.491342,0.452381,0.920705,1.572987,1.727062,0.244019
0,20190827,2,772,1079,293,0.715477,0.271548,0.379534,1.746226,1.865725,0.522184
0,20200731,1,173,546,121,0.316850,0.221612,0.699422,2.544441,1.597515,0.834711
0,20200731,2,1040,303,77,3.432343,0.254125,0.074038,2.326146,0.986308,0.168831


In [9]:
# Merge and export
final_df = matches.merge(df, on = "Cell_ID", how = "inner")
final_df.drop(["Position_x", "Experiment_x", "Timepoint_x", "Dataset_x", "index"], axis = 1, inplace = True)
final_df.rename(columns = {
    "Position_y": "Position", 
    "Experiment_y": "Experiment", 
    "Timepoint_y": "Timepoint", 
    "Dataset_y": "Dataset"
}, inplace = True)

final_df.to_csv(root + "MasterDataFrame_MatchesLineages.csv")
print("Finished exporting the final dataframe.")

Finished exporting the final dataframe.
